In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools as it
from collections import defaultdict
import seaborn as sns
pd.set_option('display.max_columns', None)

In [2]:
train_trs = pd.read_csv('/home/user/Desktop/courseProject/data/train_transaction.csv')
train_idn = pd.read_csv('/home/user/Desktop/courseProject/data/train_identity.csv')

In [3]:
data_merged = pd.merge(train_trs, train_idn, left_on='TransactionID', right_on='TransactionID', how='left')

In [4]:
def get_share_of_NaN(df):
    result = pd.DataFrame(columns=['Name', 'Number_of_NaN', 'Share_of_NaN'])
    colcount = df.count()
    length = len(df)
    for col_name in colcount.keys():
        result.loc[len(result)] = [col_name, length-colcount[col_name], (length-colcount[col_name])/length]
    return result

def remove_columns_with_many_NaN(df, max_nan_rate):
    '''
    Параметры:
    df - DataFrame
    max_nan_rate - максимальная допустимая доля NaN в колонках датафрейма
    Функция возвращает:
    1) новый датафрейм, в котором удалены колонки, в которых доля NaN болше, чем max_nan_rate
    2) список удалённых колонок
    '''
    df_copy = df.copy()
    removed_columns = []
    nan_stat = get_share_of_NaN(df)
    for i in range(len(nan_stat)):
        column = nan_stat.loc[i]
        if(column['Share_of_NaN'] > max_nan_rate):
            removed_columns.append(column['Name'])
            
    df_copy.drop(columns=removed_columns, inplace=True)
    return df_copy, removed_columns

def remove_columns_with_big_correlation(df, max_corr):
    '''
    Параметры:
    df - DataFrame
    max_corr - максимальная допустимая корреляция между колонками
    Функция возвращает:
    1) новый датафрейм, в котором удалены колонки, в которых корреляция болше, чем max_corr
    2) множество удалённых колонок
    '''
    df_copy = df.copy()
    removed_columns = set()
    corrs = df.corr()
    cols = corrs.columns
    for i in range(len(cols)):
        col_name_1 = cols[i]
        if col_name_1 in {'TransactionID', 'isFraud', 'TransactionDT'} or col_name_1 in removed_columns:
            continue
        
        for j in range(i+1, len(cols)):
            col_name_2 = cols[j]
            if abs(corrs[col_name_1][col_name_2]) > max_corr:
                removed_columns.add(col_name_2)

    df_copy.drop(columns=removed_columns, inplace=True)
    return df_copy, removed_columns

In [5]:
print(type(train_idn))
print(type(data_merged))
print(len(data_merged.columns))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
434


In [6]:
%%time
data_merged, removed_nan_cols = remove_columns_with_many_NaN(data_merged, 0.85)
data_merged_rm_cols, removed_corr_cols = remove_columns_with_big_correlation(data_merged, 0.9)

In [10]:
data_merged_rm_cols.to_csv('/home/user/Desktop/courseProject/data/train_cols.csv')

In [7]:
print(len(data_merged_rm_cols.columns))

218


In [31]:
cat_features_mask = (data_merged_rm_cols.dtypes == "object").values
val_features_mask = (data_merged.dtypes != "object").values
for i in range(len(data_merged_rm_cols.columns)):
    if data_merged_rm_cols.iloc[:,i].name in ['card1', 'card2', 'card3', 'card5', 'addr1', 'addr2']:
        cat_features_mask[i] = True
        val_features_mask[i] = False



    #real_cols = train_trs[train_trs.columns[~cat_features_mask]]
cat_cols = data_merged_rm_cols[data_merged_rm_cols.columns[cat_features_mask]]
var_cols = data_merged_rm_cols[data_merged_rm_cols.columns[~cat_features_mask]]
print(cat_cols)

       ProductCD  card1  card2  card3       card4  card5   card6  addr1  \
0              W  13926    NaN  150.0    discover  142.0  credit  315.0   
1              W   2755  404.0  150.0  mastercard  102.0  credit  325.0   
2              W   4663  490.0  150.0        visa  166.0   debit  330.0   
3              W  18132  567.0  150.0  mastercard  117.0   debit  476.0   
4              H   4497  514.0  150.0  mastercard  102.0  credit  420.0   
...          ...    ...    ...    ...         ...    ...     ...    ...   
590535         W   6550    NaN  150.0        visa  226.0   debit  272.0   
590536         W  10444  225.0  150.0  mastercard  224.0   debit  204.0   
590537         W  12037  595.0  150.0  mastercard  224.0   debit  231.0   
590538         W   7826  481.0  150.0  mastercard  224.0   debit  387.0   
590539         W  15066  170.0  150.0  mastercard  102.0  credit  299.0   

        addr2 P_emaildomain R_emaildomain   M1   M2   M3   M4   M5   M6   M7  \
0        87.0      

In [ ]:
train_merged_no_corr, big_corr_columns = remove_columns_with_big_correlation(new_train_merged, 0.9)